# TextAttack CLARE

Replication of the training, evaluating, and attacking a model using TextAttack recipe CLARE.

In [3]:
!pip install textattack[tensorflow,optional]

  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2


In [6]:
!pip install tensorflow==2.14

## Training

First, we're going to train a model. TextAttack integrates directly with [transformers](https://github.com/huggingface/transformers/) and [datasets](https://github.com/huggingface/datasets) to train any of the `transformers` pre-trained models on datasets from `datasets`.

Let's use the AG News dataset: it's relatively short, and showcases the key features of `textattack train`. Let's take a look at the dataset using `textattack peek-dataset`:

In [4]:
!textattack peek-dataset --dataset-from-huggingface ag_news

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
2023-12-17 15:55:15 INFO: Downloading default packages for language: en (English) ...
2023-12-17 15:55:24 INFO: Finished downloading models and saved to /root/stanza_resources.
2023-12-17 15:55:27.493211: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unabl

The dataset looks good! It's lowercased already, so we'll make sure our model is uncased. The longest input is 178 words, so we can cap our maximum sequence length (`--model-max-length`) at 180.

We'll train [`distilbert-base-uncased`](https://huggingface.co/transformers/model_doc/distilbert.html), since it's a relatively small model, and a good example of how we integrate with `transformers`.

So we have our command:

```bash
textattack train                      \ # Train a model with TextAttack
    --model distilbert-base-uncased   \ # Using distilbert, uncased version, from `transformers`
    --dataset rotten_tomatoes         \ # On the Rotten Tomatoes dataset
    --model-num-labels 4              \ # That has 4 labels
    --model-max-length 180             \ # With a maximum sequence length of 180
    --per-device-train-batch-size 128 \ # And batch size of 128
    --num-epochs 3                    \ # For 3 epochs
```

Now let's run it (please remember to use GPU if you have access):

In [ ]:
#ds = tfds.load('huggingface:yelp_review_full/yelp_review_full')
#distilroberta-base

In [5]:
!textattack train --model-name-or-path distilbert-base-uncased --dataset ag_news --model-num-labels 4 --model-max-length 180 --per-device-train-batch-size 128 --num-epochs 3

2023-12-17 15:56:31.379871: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 15:56:31.379936: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 15:56:31.379992: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 15:56:32.754164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-17 15:56:35.795109: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the 

## Evaluation

We successfully fine-tuned `distilbert-base-cased` for 3 epochs. Now let's evaluate it using `textattack eval`. This is as simple as providing the path to the pretrained model (that you just obtain from running the above command!) to `--model`, along with the number of evaluation samples. `textattack eval` will automatically load the evaluation data from training:

In [23]:
!textattack eval --num-examples 100 --model ./outputs/2023-12-17-15-56-36-938484/best_model/ --dataset-from-huggingface ag_news --dataset-split test

2023-12-17 21:53:37.872324: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 21:53:37.872372: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 21:53:37.872421: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 21:53:38.965688: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-17 21:53:41.714307: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the 

Awesome -- we were able to train a model up to 94.20% accuracy on the test dataset – with only a single command!

## Attack

Finally, let's attack our pre-trained model.  We can do this by passing `--recipe clare` to `textattack attack`.

> *Warning*: We're printing out 100 examples and, if the attack succeeds, their perturbations. The output of this command is going to be quite long!


In [7]:
!textattack attack --help

2023-12-17 18:15:06.960612: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 18:15:06.960668: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 18:15:06.960719: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 18:15:08.185269: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-17 18:15:11.495757: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the 

In [19]:
!textattack attack --recipe clare --num-examples 100 --model ./outputs/2023-12-17-15-56-36-938484/best_model/ --dataset-from-huggingface ag_news --dataset-split test --query-budget 5000   --enable-advance-metrics


2023-12-17 19:56:46.426617: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 19:56:46.426669: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 19:56:46.426724: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 19:56:47.543735: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-17 19:56:50.288145: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the 

Text attack did not complete due to lack of resources

Looks like our model was 94% successful (makes sense - same evaluation set as `textattack eval`!), meaning that TextAttack attacked the model with 94% examples (since the attack won't run if an example is originally mispredicted). The attack success rate was %, meaning that Clare failed to find an adversarial example only % (1 out of ) of the time.



In [17]:
!textattack attack --recipe textfooler --num-examples 100 --model ./outputs/2023-12-17-15-56-36-938484/best_model/ --dataset-from-huggingface ag_news --dataset-split test --query-budget 5000   --enable-advance-metrics


2023-12-17 19:49:07.790520: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 19:49:07.790569: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 19:49:07.790616: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 19:49:08.892406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-17 19:49:11.644213: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the 

#**Robust models**

Adversarial Training.


In [14]:
!textattack train --help

2023-12-17 19:45:43.481438: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 19:45:43.481507: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 19:45:43.481562: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 19:45:44.629501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-17 19:45:47.382016: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the 

In [16]:
!textattack train --attack clare --model-name-or-path distilbert-base-uncased --dataset ag_news --model-num-labels 4 --model-max-length 180 --per-device-train-batch-size 128 --num-epochs 3 --attack-epoch-interval 1 --num-train-adv-examples 100

2023-12-17 22:59:41.696267: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 22:59:41.696341: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 22:59:41.696381: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 22:59:43.169668: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
textattack: Loading transformers AutoModelForSequenceClassification: distilbert-base-uncased
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initi

In [14]:
!textattack train --attack textfooler --model-name-or-path distilbert-base-uncased --dataset ag_news --model-num-labels 4 --model-max-length 180 --per-device-train-batch-size 128 --num-epochs 3 --attack-epoch-interval 1 --num-train-adv-examples 100 --query-budget 5000

2023-12-17 22:57:17.051177: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 22:57:17.051251: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 22:57:17.054693: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 22:57:20.487095: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
textattack: Loading transformers AutoModelForSequenceClassification: distilbert-base-uncased
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initi